# Step one: Set up Datalab

In [ ]:
!pip install --upgrade google-cloud-bigquery

In [13]:
from google.cloud import bigquery
client = bigquery.Client()

# Step two: Create your dataset

In [ ]:
dataset = bigquery.Dataset(client.dataset('basketball_frenzy'))
dataset.location = 'US'
client.create_dataset(dataset)

# Step three: Generate your training and feature data

## Generate the input features (feature engineering)
The input features include the mean and standard deviation of previous game statistics for both home teams and away teams using different time windows. The time windows used are 10, 5 and 1 games before the current game.

[Open the feature input query](https://bigquery.cloud.google.com/savedquery/1057666841514:77711b21274b4c6485c907483ef2f6fe)

## Generate your training data
After generating the input features, we generate our training data. The training data is written to the basketball_frenzy.wide_games table. When we generate our training data, we exclude a game to use for predicting outcomes.

[Open the training data query](https://bigquery.cloud.google.com/savedquery/1057666841514:9d4ec2ed8a864e7e8d8c3c9a65faa178)

# Step four: Create your model

Next, we create a linear regression model. The model is used to predict the combined three point goal attempts based on the previous game statistics.

In [15]:
%reload_ext google.cloud.bigquery

## Create and train the model

The model is used to predict the combined three point field goal attempts based on the previous game statistics.

In [17]:
%%bigquery
CREATE OR REPLACE MODEL `basketball_frenzy.ncaa_model` OPTIONS(
  model_type='linear_reg', data_split_method='random', data_split_eval_fraction=0.1, max_iteration=50) AS
SELECT
  * EXCEPT(
    game_id, season, scheduled_date,
    total_three_points_made,
    total_three_points_att),
  total_three_points_att as label
FROM
  `basketball_frenzy.wide_games`
WHERE
  # remove the game to predict
  game_id != 'f1063e80-23c7-486b-9a5e-faa52beb2d83'

""


# Step five: Get training statistics

In [43]:
%%bigquery
SELECT
  *
FROM
  ML.TRAINING_INFO(MODEL `basketball_frenzy.ncaa_model`)

,training_run,iteration,loss,eval_loss,duration_ms,learning_rate
0,0,9,47.777819,101.602688,8937,0.10
1,0,8,55.269309,121.930399,7381,0.05
2,0,7,65.000100,123.197775,7509,0.05
3,0,6,78.562554,148.182288,10866,0.05
4,0,5,97.614917,151.837730,6750,0.05
5,0,4,124.627608,199.294534,7778,0.05
6,0,3,163.408474,209.782692,7614,0.05
7,0,2,220.265712,303.716457,8564,0.05
8,0,1,308.056427,336.275917,7909,0.05
9,0,0,525.217596,624.339335,5877,0.20


The Training Data Loss column represents the loss metric calculated after the given iteration on the training dataset. Since we performed a linear regression, this column is the mean squared error. The Evaluation Data Loss column is the same loss metric calculated on the holdout dataset (data that is held back from training to validate the model).

# Step six: Evaluate your model

In [44]:
%%bigquery
WITH eval_table AS (
SELECT
  *,
  total_three_points_att AS label
FROM
  `basketball_frenzy.wide_games` )
SELECT
  *
FROM
  ML.EVALUATE(MODEL `basketball_frenzy.ncaa_model`,
    TABLE eval_table)

,mean_absolute_error,mean_squared_error,mean_squared_log_error,median_absolute_error,r2_score,explained_variance
0,5.58347,53.113665,0.031193,4.42119,0.298546,0.453789


An important metric in the evaluation results is the mean absolute error. Mean absolute error is the average difference between the predicted label and the actual label.

# Step seven: Use our model to predict outcomes
Now that we have evaluated our model, the next step is to use the ML.PREDICT function to predict the total three point field goal attempts in the 2018 NCAA final game: Michigan versus Villanova.

In [45]:
%%bigquery
  WITH game_to_predict AS (
  SELECT
    *
  FROM
    `basketball_frenzy.wide_games`
  WHERE
    game_id='f1063e80-23c7-486b-9a5e-faa52beb2d83' )
SELECT
  truth.game_id AS game_id,
  total_three_points_att,
  predicted_total_three_points_att
FROM (
  SELECT
    game_id,
    predicted_label AS predicted_total_three_points_att
  FROM
    ML.PREDICT(MODEL `basketball_frenzy.ncaa_model`,
      table game_to_predict) ) AS predict
JOIN (
  SELECT
    game_id,
    total_three_points_att AS total_three_points_att
  FROM
    game_to_predict) AS truth
ON
  predict.game_id = truth.game_id

,game_id,total_three_points_att,predicted_total_three_points_att
0,f1063e80-23c7-486b-9a5e-faa52beb2d83,50,49.215757
